In [2]:
############   IMPORTS   ##############
import string
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
############   SCRAPING FOR DATA   ##############
playersStatsTable = []

# scrape through players on https://www.basketball-reference.com/players/a-z, exclude players after 2015
count = 0
# iterate through each letter of directory pages
for letter in string.ascii_lowercase:
    bBallReferenceUrl = "https://www.basketball-reference.com/players/" + letter
    directoryReponse = requests.get(bBallReferenceUrl)
    
    if directoryReponse.ok:
      # find rows containing info of players
      soup = BeautifulSoup(directoryReponse.text, "lxml")
      div_players = soup.find("div", {"id": "div_players"})
      table = div_players.find('tbody')
      rows = table.find_all('tr')

      # iterate through each player/row
      for row in rows:
        count += 1
        playerStatsTable = []
        isHoF = False
        if ("</a>*" in str(row)):
          isHoF = True

        playerName = row.find('a').text
        playerhref = row.find('a')["href"]
        playerLink = "https://www.basketball-reference.com" + playerhref
        year_min = int(row.find("td", {"data-stat": "year_min"}).text)
        year_max = int(row.find("td", {"data-stat": "year_max"}).text)

        # visit player's link if year_min > 1980 and year_max < 2015, add player to players
        if (year_min > 1968 and year_max < 2015):
          playerResponse = requests.get(playerLink)
          playerStatsTable.append(playerName)

          if playerResponse.ok:
            soup = BeautifulSoup(playerResponse.text, "lxml")
            # take career average for now
            div_per_game = soup.find("div", {"id": "div_per_game"})
            playerStats = div_per_game.find('tfoot').find('tr').find_all("td")
            playerStats = playerStats[4:29]

            completeStats = True
            hasEmptyStat = False

            if (len(playerStats) == 25):
              for stat in playerStats:
                if not stat.text:
                  hasEmptyStat = True
                  break
                else:
                  playerStatsTable.append(float(stat.text))
            else:
              completeStats = False

          print(isHoF)
          if (isHoF):
            playerStatsTable.append(1)
            print(row)
          else:
            playerStatsTable.append(0)

          if (completeStats and not hasEmptyStat):
            playersStatsTable.append(playerStatsTable)

In [3]:
print(np.matrix(playersStatsTable))

[['Alaa Abdelnaby' '256.0' '53.0' ... '1.9' '5.7' '0']
 ['Kareem Abdul-Jabbar' '1560.0' '625.0' ... '3.0' '24.6' '1']
 ['Mahmoud Abdul-Rauf' '586.0' '336.0' ... '1.9' '14.6' '0']
 ...
 ['Sam Young' '249.0' '52.0' ... '1.2' '5.8' '0']
 ['Wang Zhizhi' '137.0' '1.0' ... '1.1' '4.4' '0']
 ['George Zídek' '135.0' '23.0' ... '1.8' '3.4' '0']]


In [4]:
# export data to Excel sheet
df = pd.DataFrame(playersStatsTable)
df.to_excel(excel_writer = "NBAPlayersStats.xlsx")

In [104]:
# import excel
ogdf = pd.read_excel('NBAPlayersStats.xlsx')
ogdf = ogdf.iloc[: , 1:]
df = ogdf
print(df)

                       0     1    2     3     4     5      6    7    8   \
0          Alaa Abdelnaby   256   53  12.5   2.4   4.8  0.502  0.0  0.0   
1     Kareem Abdul-Jabbar  1560  625  36.8  10.2  18.1  0.559  0.0  0.0   
2      Mahmoud Abdul-Rauf   586  336  26.7   6.0  13.6  0.442  0.8  2.3   
3       Tariq Abdul-Wahad   236  145  20.4   3.1   7.3  0.417  0.1  0.3   
4     Shareef Abdur-Rahim   830  704  34.8   6.5  13.9  0.472  0.2  0.6   
...                   ...   ...  ...   ...   ...   ...    ...  ...  ...   
1744       Korleone Young     3    0   5.0   1.7   3.3  0.500  0.3  1.3   
1745        Michael Young    49    2   9.6   1.9   4.1  0.465  0.2  0.6   
1746            Sam Young   249   52  15.9   2.2   5.1  0.442  0.2  0.7   
1747          Wang Zhizhi   137    1   9.2   1.6   3.7  0.417  0.5  1.3   
1748         George Zídek   135   23   9.8   1.2   2.9  0.408  0.0  0.0   

         9   ...   17   18    19   20   21   22   23   24    25  26  
0     0.000  ...  1.1  2.2   

In [105]:
######## get Training and Testing ########
y = df.iloc[: , 26:]
X = df.iloc[: , 1:26]
# drop stats Games Played & Games Started 
cols = [0,1]
X.drop(X.columns[cols], axis=1, inplace=True)
print(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

print(len(X_train),len(X_test), len(y_train), len(y_test))

        3     4     5      6    7    8      9     10    11     12  ...     16  \
0     12.5   2.4   4.8  0.502  0.0  0.0  0.000   2.4   4.8  0.503  ...  0.701   
1     36.8  10.2  18.1  0.559  0.0  0.0  0.056  10.2  18.1  0.560  ...  0.721   
2     26.7   6.0  13.6  0.442  0.8  2.3  0.354   5.2  11.3  0.460  ...  0.905   
3     20.4   3.1   7.3  0.417  0.1  0.3  0.237   3.0   7.0  0.425  ...  0.703   
4     34.8   6.5  13.9  0.472  0.2  0.6  0.297   6.4  13.2  0.480  ...  0.810   
...    ...   ...   ...    ...  ...  ...    ...   ...   ...    ...  ...    ...   
1744   5.0   1.7   3.3  0.500  0.3  1.3  0.250   1.3   2.0  0.667  ...  1.000   
1745   9.6   1.9   4.1  0.465  0.2  0.6  0.296   1.8   3.6  0.491  ...  0.711   
1746  15.9   2.2   5.1  0.442  0.2  0.7  0.280   2.1   4.4  0.468  ...  0.742   
1747   9.2   1.6   3.7  0.417  0.5  1.3  0.385   1.0   2.4  0.435  ...  0.735   
1748   9.8   1.2   2.9  0.408  0.0  0.0  0.250   1.2   2.9  0.409  ...  0.783   

       17   18    19   20  

In [106]:
## Over-sample data to resolve isHofF vs !isHofF class imabalance ##
dfCopy = df
class_count_0, class_count_1 = dfCopy.iloc[:,-1:].value_counts()

# import library
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
X_ros, y_ros = ros.fit_resample(X, y)

print('Original X shape', X.shape)
print('Resample X shape', X_ros.shape)
print('Original y shape', y.shape)
print('Resample y shape', y_ros.shape)

print(y.value_counts())
print(y_ros.value_counts())

# split ros
X_train,X_test,y_train,y_test = train_test_split(X_ros, y_ros,test_size=0.2)

print(len(X_train),len(X_test), len(y_train), len(y_test))
print(X_train.shape)
print(y_train.shape)

Original X shape (1749, 23)
Resample X shape (3358, 23)
Original y shape (1749, 1)
Resample y shape (3358, 1)
26
0     1679
1       70
dtype: int64
26
0     1679
1     1679
dtype: int64
2686 672 2686 672
(2686, 23)
(2686, 1)


In [22]:
######## Train Neural Network Model ########
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import SGD
from keras.layers import Input
from keras.models import Model
from tensorflow.keras import optimizers
import keras

# NN Model
# NN_Model = Sequential()
# NN_Model.add(Dense(1, input_shape=(24,), activation='sigmoid'))

inputs = Input(shape=(23,), name="stats")
x = Dense(64, activation="relu", name="dense_1")(inputs)
x = Dense(64, activation="relu", name="dense_2")(x)
outputs = Dense(2, activation="softmax", name="predictions")(x)

NN_Model = Model(inputs=inputs, outputs=outputs)

NN_Model.compile(
    # Optimizer
    optimizer=optimizers.RMSprop(),
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

NN_Model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
84/84 [==============================] - 1s 2ms/step - loss: 0.4175 - sparse_categorical_accuracy: 0.8313
Epoch 2/200
84/84 [==============================] - 0s 2ms/step - loss: 0.2685 - sparse_categorical_accuracy: 0.8984
Epoch 3/200
84/84 [==============================] - 0s 2ms/step - loss: 0.2141 - sparse_categorical_accuracy: 0.9203
Epoch 4/200
84/84 [==============================] - 0s 2ms/step - loss: 0.2006 - sparse_categorical_accuracy: 0.9203
Epoch 5/200
84/84 [==============================] - 0s 2ms/step - loss: 0.1796 - sparse_categorical_accuracy: 0.9293
Epoch 6/200
84/84 [==============================] - 0s 2ms/step - loss: 0.1795 - sparse_categorical_accuracy: 0.9319
Epoch 7/200
84/84 [==============================] - 0s 2ms/step - loss: 0.1640 - sparse_categorical_accuracy: 0.9315
Epoch 8/200
84/84 [==============================] - 0s 2ms/step - loss: 0.1532 - sparse_categorical_accuracy: 0.9509
Epoch 9/200
84/84 [==============================] - 0s 

In [112]:
# Test
score = NN_Model.evaluate(X_test, y_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
pred = NN_Model.predict(X_test)[:,1]
pred[pred > 0.5] = 1
pred[pred <= 0.5] = 0
 
#calculating precision and reall
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
print("precision: ")
print(precision)
print("recall: ")
print(recall)
confusion_matrix(y_test, pred)

21/21 [==============================] - 0s 2ms/step
precision: 
0.9736070381231672
recall: 
1.0


array([[331,   9],
       [  0, 332]])

In [ ]:
# Using customized current players
LukaDoncic = "33.8	9.0	19.7	.457	2.8	8.2	.337	6.2	11.5	.543	.528	5.6	7.6	.737	1.0	7.5	8.5	8.0	1.1	0.4	4.1	2.2 26.4"
KobeBryant = "36.1	8.7	19.5	.447	1.4	4.1	.329	7.3	15.3	.479	.482	6.2	7.4	.837	1.1	4.1	5.2	4.7	1.4	0.5	3.0	2.5	25.0"
ShaqOneal = "34.7	9.4	16.1	.582	0.0	0.0	.045	9.4	16.1	.583	.582	4.9	9.3	.527	3.5	7.4	10.9	2.5	0.6	2.3	2.7	3.4	23.7"
CarmeloAnthony = "34.5	8.0	18.0	.447	1.4	3.9	.355	6.7	14.1	.472	.485	5.0	6.2	.814	1.6	4.6	6.2	2.7	1.0	0.5	2.4	2.8	22.5"
JRSmith = "26.9	4.5	10.7	.419	2.0	5.3	.373	2.5	5.4	.463	.511	1.5	2.0	.733	0.5	2.6	3.1	2.1	1.0	0.2	1.3	2.2	12.4"
LeBronJames = "38.2	9.9	19.6	.505	1.6	4.5	.346	8.3	15.1	.552	.545	5.7	7.8	.734	1.2	6.3	7.5	7.4	1.6	0.8	3.5	1.8	27.1"
RJBarret = "33.5	6.3	15.1	.418	1.6	4.6	.357	4.7	10.5	.445	.472	3.3	4.7	.697	0.9	4.6	5.6	2.9	0.8	0.3	2.1	2.3	17.5"
JeremyLin = "25.5	3.9	9.1	.433	0.9	2.7	.342	3.0	6.3	.473	.485	2.8	3.5	.809	0.4	2.4	2.8	4.3	1.1	0.4	2.3	2.2	11.6"
DerrickRose = "31.7	7.1	15.6	.457	0.8	2.6	.316	6.3	13.0	.485	.483	3.1	3.8	.830	0.8	2.5	3.3	5.4	0.8	0.3	2.5	1.3	18.2"
KristapsPorzingis = "30.8	6.8	15.3	.444	1.8	5.0	.353	5.0	10.2	.488	.502	3.6	4.4	.820	1.7	6.1	7.9	1.6	0.7	1.9	1.6	3.0	18.9"
EvanFournier = "28.6	5.1	11.5	.445	2.0	5.3	.381	3.1	6.3	.499	.532	2.0	2.5	.798	0.4	2.3	2.7	2.6	0.9	0.2	1.6	2.4	14.3"
ImmanuelQuickley = "21.4	3.6	9.2	.393	1.8	4.9	.365	1.8	4.3	.426	.491	2.3	2.6	.885	0.4	2.3	2.7	2.8	0.6	0.1	1.1	2.0	11.4"
JalenBrunson = "24.7	4.6	9.3	.494	1.0	2.7	.373	3.6	6.6	.544	.548	1.6	2.0	.800	0.4	2.6	3.0	3.7	0.6	0.0	1.3	1.7	11.9"

playerNames = [
    "Luka Doncic",
    "Carmelo Anthony",
    "JR Smith",
    "LeBron James",
    "Jeremy Lin",
    "Kristaps Porzingis",
    "Jalen Brunson",
    "Evan Fournier",
    "Immanuel Quickley",
    "Julius Randle",
    "Cam Reddish",
    "Derrick Rose",
    "Obi Toppin",
    "Stephen Curry",
    "Joakim Noah",
    "Draymond Green",
    "JaVale McGee",
    "Danilo Gallinari",
    "De'Aaron Fox",
    "Vince Carter",
    "Travis Knight",
    "Patrick Beverley",
    "José Calderón",
    "Tyson Chandler",
    "Amar'e Stoudemire"
]

playersStats = [
    "33.8	9.0	19.7	.457	2.8	8.2	.337	6.2	11.5	.543	.528	5.6	7.6	.737	1.0	7.5	8.5	8.0	1.1	0.4	4.1	2.2 26.4",
    "34.5	8.0	18.0	.447	1.4	3.9	.355	6.7	14.1	.472	.485	5.0	6.2	.814	1.6	4.6	6.2	2.7	1.0	0.5	2.4	2.8	22.5",
    "26.9	4.5	10.7	.419	2.0	5.3	.373	2.5	5.4	.463	.511	1.5	2.0	.733	0.5	2.6	3.1	2.1	1.0	0.2	1.3	2.2	12.4",
    "38.2	9.9	19.6	.505	1.6	4.5	.346	8.3	15.1	.552	.545	5.7	7.8	.734	1.2	6.3	7.5	7.4	1.6	0.8	3.5	1.8	27.1",
    "25.5	3.9	9.1	.433	0.9	2.7	.342	3.0	6.3	.473	.485	2.8	3.5	.809	0.4	2.4	2.8	4.3	1.1	0.4	2.3	2.2	11.6",
    "30.8	6.8	15.3	.444	1.8	5.0	.353	5.0	10.2	.488	.502	3.6	4.4	.820	1.7	6.1	7.9	1.6	0.7	1.9	1.6	3.0	18.9",
    "24.7	4.6	9.3	.494	1.0	2.7	.373	3.6	6.6	.544	.548	1.6	2.0	.800	0.4	2.6	3.0	3.7	0.6	0.0	1.3	1.7	11.9",
    "28.6	5.1	11.5	.445	2.0	5.3	.381	3.1	6.3	.499	.532	2.0	2.5	.798	0.4	2.3	2.7	2.6	0.9	0.2	1.6	2.4	14.3",
    "21.4	3.6	9.2	.393	1.8	4.9	.365	1.8	4.3	.426	.491	2.3	2.6	.885	0.4	2.3	2.7	2.8	0.6	0.1	1.1	2.0	11.4",
    "31.2	6.5	13.9	.472	0.9	2.6	.332	5.7	11.3	.504	.503	3.8	5.1	.743	2.0	7.3	9.3	3.6	0.7	0.5	2.7	3.1	17.7",
    "24.9	3.6	9.2	.387	1.4	4.2	.325	2.2	5.0	.439	.461	2.0	2.4	.841	0.6	2.6	3.2	1.3	1.1	0.4	1.4	2.0	10.5",
    "31.7	7.1	15.6	.457	0.8	2.6	.316	6.3	13.0	.485	.483	3.1	3.8	.830	0.8	2.5	3.3	5.4	0.8	0.3	2.5	1.3	18.2",
    "14.3	2.6	5.1	.521	0.6	1.9	.307	2.1	3.2	.647	.578	0.8	1.1	.753	0.7	2.3	3.0	0.8	0.3	0.4	0.6	1.2	6.7",
    "34.3	8.3	17.6	.473	3.8	8.8	.428	4.5	8.8	.520	.581	3.9	4.3	.908	0.7	4.0	4.6	6.5	1.7	0.2	3.1	2.4	24.3",
    "27.7	3.3	6.8	.491	0.0	0.0	.000	3.3	6.8	.493	.491	2.1	2.9	.700	3.2	5.8	9.0	2.8	0.8	1.3	1.7	2.8	8.8",
    "28.5	3.2	7.2	.441	0.8	2.6	.315	2.3	4.6	.514	.499	1.5	2.1	.712	1.1	5.8	6.9	5.4	1.4	1.0	2.2	2.8 8.7",
    "17.0	3.4	5.9	.576	0.0	0.1	.182	3.4	5.8	.582	.577	1.1	1.9	.605	1.9	3.4	5.3	0.4	0.4	1.5	1.0	2.2	8.0",
    "29.9	4.7	11.1	.428	2.0	5.1	.382	2.8	6.0	.467	.516	4.1	4.7	.877	0.7	4.1	4.8	1.9	0.7	0.4	1.2	1.8	15.6",
    "32.1	7.0	15.3	.462	1.1	3.5	.320	5.9	11.7	.504	.499	3.9	5.3	.725	0.5	3.0	3.5	6.2	1.3	0.4	2.8	2.7	19.1",
    "30.1	6.0	13.8	.435	1.5	4.0	.371	4.5	9.8	.462	.489	3.1	3.9	.798	1.1	3.2	4.3	3.1	1.0	0.6	1.7	2.6	16.7",
    "12.3	1.4	3.2	.438	0.0	0.2	.259	1.4	3.1	.447	.444	0.6	0.8	.696	1.2	1.9	3.1	0.6	0.4	0.6	0.7	2.2	3.4",
    "27.4	3.1	7.5	.414	1.6	4.2	.378	1.5	3.3	.460	.520	1.1	1.4	.756	1.1	3.1	4.3	3.5	1.1	0.5	1.3	3.1	8.8",
    "26.4	3.4	7.2	.472	1.0	2.5	.407	2.4	4.7	.507	.543	1.0	1.2	.873	0.3	2.1	2.4	5.8	0.8	0.1	1.5	1.6	8.9",
    "27.3	3.1	5.1	.597	0.0	0.0	.000	3.1	5.1	.598	.597	2.1	3.2	.644	3.0	6.0	9.0	0.8	0.5	1.2	1.4	2.8	8.2",
    "1.0	7.0	13.0	.537	0.0	0.2	.236	7.0	12.9	.540	.538	4.9	6.4	.761	2.4	5.4	7.8	1.2	0.8	1.2	2.3	3.2	18.9",
]

for i in range(0, len(playerNames)):
  playerStats = playersStats[i]
  print(playerNames[i] + ": " + predictPlayer(convertToDf(playerStats)))

def convertToDf(playerStats):
  playerStats = playerStats.split()
  playerStats = np.array(playerStats)
  playerStats = pd.DataFrame(data=playerStats).T.astype(float)
  return playerStats

def predictPlayer(playerStats):
  prediction = NN_Model.predict(playerStats, verbose=0).item(1)
  if (prediction > 0.5):
    return "Future Hall of Famer"
  else:
    return "Not Future Hall of Famer"

In [60]:
# k nearest neighbors
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=2)
knn_model.fit(X_test, y_test)

KNeighborsRegressor(n_neighbors=2)

In [67]:
print(knn_model.score(X_test, y_test))
knn_model.predict(playerStats)

0.9806540764624597


array([[1.]])